In [1]:
import re, json
import pandas as pd

In [ ]:
!python -m WikiExtractor data/ptwiki-latest-pages-articles.xml.bz2 --json

In [6]:
TAG_RE = re.compile(r'<[^>]+>')


def remove_tags(text):
    return TAG_RE.sub('', text)

# Função para remover quaisquer pontuações e caracteres especiais
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

def save_txt(text, file):
    f = open(file, 'a')
    f.write(text)
    f.close()

In [2]:
with open('text/wiki.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

In [3]:
df_wiki = pd.DataFrame(data)

In [4]:
df_wiki = df_wiki.drop('url', axis=1)
df_wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057059 entries, 0 to 1057058
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1057059 non-null  object
 1   title   1057059 non-null  object
 2   text    1057059 non-null  object
dtypes: object(3)
memory usage: 24.2+ MB


In [7]:
df_wiki['text'] = df_wiki['text'].apply(preprocess_text)

In [8]:
df_wiki['text'].sample(frac=.5).to_csv('data/texto-sm.csv', index=False)
df_wiki['text'].sample(frac=.25).apply(preprocess_text).to_csv('data/texto-md.csv', index=False)
df_wiki['text'].to_csv('data/texto-lg.csv', index=False)
print('Exportado')

Exportado


In [ ]:
# df_wiki['text'].sample(frac=.5).apply(save_txt, file='data/texto-sm.txt')
# df_wiki['text'].sample(frac=.25).apply(save_txt, file='data/texto-md.txt')
# df_wiki['text'].apply(save_txt, file='data/texto-lg.txt')
# print('Exportado')